In [5]:
# =========================================== 1.6
# 1. Import thư viện
# ===========================================
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import mean_squared_error, mean_absolute_error

# ===========================================
# 2. Đọc dữ liệu
# ===========================================
file_path = r"C:\DATA\data-ecom_quangnv.csv"  
df = pd.read_csv(file_path)

texts = df["Review_VN"].astype(str).tolist()
labels = df["Category"].astype(str).tolist()

# Tokenizer & LabelEncoder
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
X = pad_sequences(tokenizer.texts_to_sequences(texts), maxlen=50, padding='post')

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)
y = to_categorical(y)

# Chia train/val giống khi huấn luyện
split = int(0.8 * len(X))
X_train, X_val = X[:split], X[split:]
y_train, y_val = y[:split], y[split:]

# ===========================================
# 3. Load model1 và model2
# ===========================================
model1 = tf.keras.models.load_model("model1_lstm_ecom.h5")
model2 = tf.keras.models.load_model("model2.h5")

# ===========================================
# 4. Dự đoán trên tập validation
# ===========================================
y_pred1 = model1.predict(X_val)
y_pred2 = model2.predict(X_val)

# ===========================================
# 5. Tính MSE, MAE, RMSE cho từng model
# ===========================================
mse1 = mean_squared_error(y_val, y_pred1)
mae1 = mean_absolute_error(y_val, y_pred1)
rmse1 = np.sqrt(mse1)

mse2 = mean_squared_error(y_val, y_pred2)
mae2 = mean_absolute_error(y_val, y_pred2)
rmse2 = np.sqrt(mse2)

# Tổng hợp kết quả
results = pd.DataFrame({
    'Metric': ['MSE', 'MAE', 'RMSE'],
    'Model1_NoDropout': [mse1, mae1, rmse1],
    'Model2_Dropout0.2': [mse2, mae2, rmse2]
})

print("\n📊 So sánh sai số giữa model1 và model2:")
print(results)

# ===========================================
# 6. Visualization so sánh sai số
# ===========================================
plt.figure(figsize=(8,5))
metrics = ['MSE', 'MAE', 'RMSE']
x = np.arange(len(metrics))
width = 0.35

plt.bar(x - width/2, results['Model1_NoDropout'], width, label='Model1 (No Dropout)')
plt.bar(x + width/2, results['Model2_Dropout0.2'], width, label='Model2 (Dropout 0.2)')

plt.xticks(x, metrics)
plt.ylabel('Giá trị sai số')
plt.title('So sánh MSE / MAE / RMSE giữa Model1 và Model2')
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# ===========================================
# 7. Kết luận nhanh
# ===========================================
if mse2 < mse1 and mae2 < mae1:
    print("\n✅ Model2 (Dropout) tổng thể có sai số thấp hơn → generalization tốt hơn, ít overfitting hơn.")
else:
    print("\n⚠️ Model1 (No Dropout) có thể overfit hơn hoặc chưa regularize đủ.")


313/313 ━━━━━━━━━━━━━━━━━━━━ 13s 36ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 12s 38ms/step


ValueError: y_true and y_pred have different number of output (3!=5)